# Cross-matching astronomical catalogs

Cross-matching is the process of finding entries (objects) in two or more tables (catalogs) considered, in reality, to be the same.

Now, for a longer explanation, consider two astronomical catalogs -- where rows contain properties of  astronomical objects, columns organize those properties and each (real) astronomical object can be present no more than once in each catalog. To picture a very simple situation, without loosing in generality, ee can think of such catalogs as being the products of optical and x-ray observations of a certain region of the sky. It it is expected -- say the *null-hypothesis* -- that not all but some of the objects are in both catalogs. Notice, though, that the catalogs -- as data structures -- are not required to share any other structural property like number or order of rows and columns. This is a typical scenario astronomers handle a cross-matching.

The process of finding the objects shared by both catalogs is called cross-matching. In extra-galactic astronomy, in practice the objects -- galaxies, QSOs -- do not move; which brings their position in the sky to be used as an identifier. The very basic parameters to be used for the cross-matching is then Right Ascension and Declination: at each of those catalogs, the objects that are in the same position of the sky are said to be the same. The result of this process is a *cross-matched* catalog, containing the matched objects and the merge of columns (*i.e.*, properties) from both catalogs.

Although it looks like a simple subject, cross-matching is a long-standing issue in astronomy. And it is quite easy to see that once we realize how observational effects (*e.g.*, astronomical seeing) affect, for instance, an object's position measurement. The uncertainties added to the measurements cause the same object to show up slightly different positions in each catalog; given that we have to match not exact values, but coordinates that should match within a tolerance value |egr| -- also called *error radius*  or *search radius*. Intrinsic astrophysical effects can also cause the same astronomical object not to match between catalogs of different wavebands, for example, Radio Lobes generated by an Active Galactic Nuclei (AGN) can cause a mismatch between a radio and an optical catalogs.

In practice, the basic idea on cross-matching is to associate the closest sources within a search radius. Which is called a *pure positional* method. Other two association methods can be found in the literature, *maximum likelihood* [SeS92]_ and *bayesian statistics* [BeS08]_.

In this work the maximum likelihood estimator (MLE), great circle (GC) and nearest neighbout (NN)
were implemented.

.. [SeS92] "*On the likelihood ratio for source identification*", W. Sutherland and W. Saunders, MNRAS, 1992
.. [BeS08] "*Probabilistic Cross Identification of Astronomical Sources*", T. Budavari and A. Szalay, ApJ, 2008

.. |egr|  unicode:: U+003B5 .. GREEK SMALL LETTER EPSILON


## Nearest Neighbor algorithm

The algorithm implement for cross-matching catalogs is based on a *nearest neighbor* search.

### Two mock catalogs

Consider two catalogs **A**, **B**. The algorithm does a two-way search, first taking catalog *A* as the *reference catalog* and searching for the nearest neighbor(NN) in *B* (*B->A*), then for the NN of *B* in *A* (*A->B*). Because there can be multiple matches with the same NN (different sources in *A* can have the same object in *B* as NN, when *B->A* for example) a cleaning step is necessary to remove this multiplicity. Such cleaning is done by keeping only the pair of objects (*i.e.*, match) that exist on both matching lists.

`Figure 1`_ illustrates a spatial distribution of objects from two different catalogs -- *A*,*B*. This image should help us in visualizing the matching algorithm. The size of the circles symbolizes the *error radius* on object positions, they all measure ``20 pixels``.

.. figure:: images/xmatch/TOY_artificial_obj_distro_2_explain_matchAB_labeled.png
   :name:     Figure 1
   :scale:    75%
   :align:    center

   Mock spatial distribution of objects. *Blue* and *red* to mimic catalogs "A" and "B".


The search for neighbors
========================

Searching for the nearest neighbor to the objects in catalog *A* we find the objects in *B* as listed on the  table `Matching (NN) B->A`_ below. On the other hand, looking for the NN in the other direction, *B* as reference and looking for neighbors in *A*, we have the matches shown in `Matching (NN) A->B`_. As we might already been expecting, multiple objects were found from the second ("neighborhood") catalog to match.

Next step is to clean those multiplicities. Going through the *multiple matches* given from both tables we should agree that the real *nearest neighbors* are:

* **B->A**: [A:3,B:1], [A:7,B:4], [A:4,B:2], [A:12,B:5]
* **A->B**: [B:2,A:4]

That is to say that all other multiple matches should be taken out. Doing such, we end up with `Table 3`_, where separation (in pixels) are shown, where indeed only those 4 matchings-pairs are kept for the final result.

Notice, though, that we are not get into the question whether the matches are right or wrong; not yet. The current discussion regards only the a search for a unique set of *nearest neighbors* pairs as a basic idea to establish before going further on qualifying those matches.

.. table:: Matching (NN) B->A
   :name:  Matching (NN) B->A

   ==== ==== ==== =======
   A_ID   x    y  NN_in_B
   ==== ==== ==== =======
     1  301   34        1
     2   51   74        1
     3  230  145        1
     4  404  232        2
     5  229  265        4
     6   52  286        4
     7   83  288        4
     8  346  289        2
     9  317  339        2
    10  214  376        4
    11  465  388        5
    12  401  455        5
   ==== ==== ==== =======


.. table:: Matching (NN) A->B
   :name:  Matching (NN) A->B

   ==== ==== ==== =======
   B_ID   x    y  NN_in_A
   ==== ==== ==== =======
     1  299  126        3
     2  391  246        4
     3  445  249        4
     4  120  359        7
     5  428  441       12
     6  125  446       10
   ==== ==== ==== =======


.. table:: NN matching-table
   :name:  Table 3

   ===== ==== ==== ==== ==== ==== =====
    A_ID  x    y   B_ID  x    y   dist
   ===== ==== ==== ==== ==== ==== =====
      1   301   34  nan  nan  nan   nan
      2    51   74  nan  nan  nan   nan
      3   230  145   1   299  126    71
      4   404  232   2   391  246    19
      5   229  265  nan  nan  nan   nan
      6    52  286  nan  nan  nan   nan
      7    83  288   4   120  359    80
      8   346  289  nan  nan  nan   nan
      9   317  339  nan  nan  nan   nan
     10   214  376  nan  nan  nan   nan
     11   465  388  nan  nan  nan   nan
     12   401  455   5   428  441    30
   ===== ==== ==== ==== ==== ==== =====


Pseudocode
----------

The (pseudocode) inputs are ``catalog-A`` and ``catalog-B``, tables (e.g, FITS files) where at least the columns ``RA`` (Right Ascension), ``DEC`` (Declination) and ``OBJID`` (Object ID, unique identifier) are present [*]_.

```
SkyCoord := ~astropy.coordinates.SkyCoord

A <- read columns 'RA','DEC','OBJID' from catalog-A
B <- read columns 'RA','DEC','OBJID' from catalog-B

coords_A <- build SkyCoord objects from 'RA','DEC'
coords_B <- build SkyCoord objects from 'RA','DEC'

match_A <- find nearest-neighbor from A in B
match_B <- find nearest-neighbor from B in A

match_AB <- filter intersection matches between match_A & match_B

matched_catalog <- join A,B following match_AB
```

The output, ``matched_catalog``, provides all columns in ``A`` and ``B`` -- for instance, ``RA``, ``DEC``, ``OBJID`` -- plus the angular separation -- ``distance_AB`` -- between the matches. The number of rows is the same as catalog A.
Using *relational database* jargon, ``matched_catalog`` is the result of a *left join* between ``A`` and ``B``.

+--------+-------+----+-----+-------+----+-----+------+
|        | A                | B                |  AB  |
+--------+-------+----+-----+-------+----+-----+------+
| index  | OBJID | RA | DEC | OBJID | RA | DEC | dist |
+========+=======+====+=====+=======+====+=====+======+
| 1      |       |    |     |       |    |     |      |
+--------+-------+----+-----+-------+----+-----+------+
| 2      |       |    |     |       |    |     |      |
+--------+-------+----+-----+-------+----+-----+------+
| ...    |       |    |     |       |    |     |      |
+--------+-------+----+-----+-------+----+-----+------+
|length A|       |    |     |       |    |     |      |
+--------+-------+----+-----+-------+----+-----+------+


.. [*] The pipeline does not consider the use of positional errors so far.



Defining a search-radius
========================

So far, the "tolerance radius" we `previously talked <xmatch_introduction>`_ about has not been considered.
On not considering this aspect, we might end up with matches between ("nearest-neighbors") objects that are not actually close to each other. In our current mock example, it can be argued that among all the possible matches `Table 3`_ only *[A:4,B:2]* and *[A:12,B:5]* do represent real matches -- if we consider ``30 pixels`` to be sufficiently close.

Although not used ever so far, the objects in our mock catalogs have an "error radius" of ``20 pixels``. Considering the errors between "A" and "B" to be independent, their intersection is given by their sum in quadrature,

.. math::

   \epsilon &= \sqrt{err_A^2 + err_B^2} \\
            &= \sqrt{20^2 + 20^2} \\
            &= 28.2

, which gives us a tolerance radius of :math:`28` pixels. Using this value to cut out "false matchings" leaves us with the pair *[A:4,B:2]*. As such, our final matching table is given by `Table 4`_ below.

.. table:: Final matching-table
   :name:  Table 4

   ===== ==== ==== ==== ==== ==== =====
    A_ID  x    y   B_ID  x    y   dist
   ===== ==== ==== ==== ==== ==== =====
      1   301   34  nan  nan  nan  nan
      2    51   74  nan  nan  nan  nan
      3   230  145  nan  nan  nan  nan
      4   404  232   2   391  246   19
      5   229  265  nan  nan  nan  nan
      6    52  286  nan  nan  nan  nan
      7    83  288  nan  nan  nan  nan
      8   346  289  nan  nan  nan  nan
      9   317  339  nan  nan  nan  nan
     10   214  376  nan  nan  nan  nan
     11   465  388  nan  nan  nan  nan
     12   401  455  nan  nan  nan  nan
   ===== ==== ==== ==== ==== ==== =====


## Great circle algorithm

The fundamental characteristic of a great-circle (gc) cross-matching is the
restriction on the region to search for counterparts, and that limiting region
is defined by a *radius* parameter around each target source.

Whereas a *nn* algorithm will *always* find a counterpart for each and every
target, the matching pair may be separated by a unreasonable distance.
The *gc* algorithm on the other hand, by restricting the search region, may not
find a counterpart for each target; and depending on the size of the search
region, it may find multiple counterparts, to decide which one to take is a
second step.

Typically, in the simplest scenario, the *nearest neighbour* is taken among the
counterpart candidates found by the *great-circle* algorithm.

The great circle approach provides a better performant algorithm (O(N*logN))
and provide a natural mechanism to sub-sample the datasets.
Such sampling mechanism may then be used to dynamically define the best match.
